# 🏥 Medical Diagnosis - ULTRA SIMPLE VERSION

**No custom theme - Pure CSS approach**

In [ ]:
!pip install datasets scikit-learn pandas numpy joblib gradio -q

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import joblib
import gradio as gr
import json
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print("✅ Ready!")

In [ ]:
# Train model
dataset = load_dataset("gretelai/symptom_to_diagnosis")
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

X_train, y_train = train_df['input_text'].values, train_df['output_text'].values
X_test, y_test = test_df['input_text'].values, test_df['output_text'].values

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,3), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

best_model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE, class_weight='balanced')
best_model.fit(X_train_tfidf, y_train)
acc = accuracy_score(y_test, best_model.predict(X_test_tfidf))

joblib.dump(best_model, 'best_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
metadata = {'model_name': 'Logistic Regression', 'accuracy': float(acc), 'classes': list(best_model.classes_), 'n_features': len(vectorizer.get_feature_names_out())}
with open('model_metadata.json', 'w') as f:
    json.dump(metadata, f)

print(f"✅ Model ready! Accuracy: {acc:.4f}")

In [ ]:
# Prediction
def predict(symptoms, num=5):
    if not symptoms or len(symptoms.strip()) < 10:
        return "⚠️ Please provide detailed symptoms", [], ""
    try:
        vec = vectorizer.transform([symptoms])
        probas = best_model.predict_proba(vec)[0]
        classes = best_model.classes_
        top = np.argsort(probas)[::-1][:num]
        text = "## Results\n\n"
        table = []
        for i, idx in enumerate(top, 1):
            diag = classes[idx].title()
            prob = probas[idx] * 100
            conf = "High" if prob > 60 else "Medium" if prob > 30 else "Low"
            text += f"**{i}. {diag}**: {prob:.1f}% ({conf})\n\n"
            table.append([i, diag, f"{prob:.1f}%", conf])
        text += "\n---\n**Note**: Always consult a physician. Educational tool only."
        ts = datetime.now().strftime("%Y-%m-%d %H:%M")
        hist = f"[{ts}] {classes[top[0]].title()} ({probas[top[0]]*100:.1f}%)\n"
        return text, table, hist
    except Exception as e:
        return f"Error: {e}", [], ""

EXAMPLES = [
    "Persistent headache, photosensitivity, nausea, visual disturbances",
    "Productive cough, dyspnea, fever, fatigue, thick sputum",
    "Joint swelling, morning stiffness, difficulty walking",
    "High fever, rigors, diaphoresis, myalgia, headache",
]

In [ ]:
# CSS only - no custom theme
css = """
/* Force EVERYTHING white and black */
.gradio-container, body, .contain, .block, .wrap {
    background: white !important;
}

/* ALL text black */
*, p, span, div, label, h1, h2, h3 {
    color: black !important;
}

/* Markdown black */
.gr-prose, .gr-prose * {
    color: black !important;
}

/* Inputs white */
textarea, input {
    background: white !important;
    color: black !important;
    border: 2px solid #999 !important;
}

/* Table */
table, th, td {
    background: white !important;
    color: black !important;
}

th {
    background: #f0f0f0 !important;
    font-weight: 700 !important;
}

/* Sharp corners */
* {
    border-radius: 0 !important;
}
"""

In [ ]:
# Interface - DEFAULT theme with CSS only
with gr.Blocks(css=css, theme=gr.themes.Default(), title="Medical Diagnosis") as demo:
    
    gr.Markdown(
        """
        # 🏥 Medical Diagnosis System
        
        ### ⚠️ Educational Tool - Not for Real Diagnosis
        Always consult a qualified physician.
        
        ---
        """
    )
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Enter Symptoms")
            symptoms = gr.Textbox(
                label="Describe symptoms",
                placeholder="Example: headache, nausea, fever...",
                lines=6
            )
            num = gr.Slider(1, 10, 5, step=1, label="Number of results")
            
            with gr.Row():
                analyze = gr.Button("Analyze", variant="primary")
                clear = gr.Button("Clear")
            
            gr.Markdown("### Examples")
            gr.Examples(EXAMPLES, symptoms)
        
        with gr.Column():
            gr.Markdown("### Results")
            results = gr.Markdown("*Awaiting analysis...*")
            table = gr.Dataframe(
                headers=["Rank", "Diagnosis", "Probability", "Confidence"],
                label="Summary"
            )
    
    with gr.Accordion("History", open=False):
        history = gr.Textbox(lines=6, interactive=False)
    
    gr.Markdown(f"""
    ---
    **Model**: {metadata['model_name']} | **Accuracy**: {metadata['accuracy']:.3f}
    """)
    
    def update_hist(curr, new):
        return curr + new if new else curr
    
    analyze.click(predict, [symptoms, num], [results, table, history]).then(
        update_hist, [history, history], history
    )
    clear.click(lambda: ("", "*Awaiting...*", [], ""), outputs=[symptoms, results, table, history])

print("✅ Interface ready!")

In [ ]:
# Launch
demo.launch(share=False, inline=True, show_error=True)